In [ ]:
! pip install  -r requirements.txt

In [1]:
import tensorflow as tf
import torch
from VideoSwinTransformer import model_configs , SwinTransformer3D_pt
from collections import OrderedDict
import numpy as np


In [9]:
tf_model = tf.keras.models.load_model('tf_weights/swin_tiny_patch244_window877_kinetics400_1k_tf.pth')

In [2]:
x_pt = torch.rand( (1,3, 8,224,224)) * 255
x_np = x_pt.numpy()
x_tf = tf.convert_to_tensor(x_np)
x_tf = tf.transpose(x_tf, perm=(0,2,3,4,1))
x_tf = tf.cast(x_tf, dtype= tf.float32)

x_tf.shape

TensorShape([1, 8, 224, 224, 3])

In [3]:
cfg_method = model_configs.MODEL_MAP["swin_tiny_patch244_window877_kinetics400_1k"]
cfg = cfg_method()

name = cfg["name"]
link = cfg['link']
del cfg["name"]
del cfg['link']

pt_model = SwinTransformer3D_pt(**cfg)

checkpoint = torch.load(f'{name}.pth')


new_state_dict = OrderedDict()
for k, v in checkpoint['state_dict'].items():
    if 'backbone' in k:
        name = k[9:]
        new_state_dict[name] = v 

pt_model.load_state_dict(new_state_dict) 

c:\Python\Python396\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


<All keys matched successfully>

In [10]:
y = tf_model.predict(x_tf)


In [23]:
z = pt_model(x_pt)

In [24]:
y = np.round(y, 4)

In [25]:

z.shape , y.shape


(torch.Size([1, 768, 4, 7, 7]), (1, 768, 4, 7, 7))

In [26]:
y[0][0][0]

array([[ 0.248 ,  0.8429,  0.0625,  0.2547,  0.0038,  0.7321,  0.3114],
       [ 0.011 ,  0.8426,  0.9979,  0.286 ,  0.4211,  0.7433, -0.0207],
       [ 0.2046,  0.4651, -0.0248,  0.2616, -0.0525, -0.038 ,  0.2936],
       [ 0.0567,  0.2358, -0.0883,  0.2856, -0.0103,  0.1148, -0.0598],
       [-0.0037,  0.6302, -0.0902, -0.112 ,  0.0935,  0.0058, -0.0441],
       [ 0.244 ,  0.118 , -0.043 , -0.0969,  0.2323,  0.2088,  0.104 ],
       [-0.1388, -0.0188,  0.2547, -0.101 ,  0.2865,  0.7858,  0.2629]],
      dtype=float32)

In [27]:
z[0][0][0]

tensor([[ 0.1783,  0.9666,  0.1493,  0.1831,  0.2065,  0.3020,  0.2416],
        [ 0.0587,  0.5428,  0.9314,  0.2074,  0.3606,  0.2332, -0.0233],
        [ 0.1550,  0.0723, -0.0436,  0.1668, -0.0259, -0.0210,  0.2261],
        [ 0.0297,  0.0195,  0.0520,  0.2142, -0.0212, -0.0434, -0.0255],
        [ 0.0119,  0.8422, -0.0031, -0.0719,  0.0650, -0.0663, -0.0116],
        [ 0.1824,  0.1240, -0.0262, -0.0766,  0.5365, -0.0499,  0.0837],
        [-0.1141, -0.0665,  0.1776, -0.1135,  0.2927,  0.2539,  0.2005]],
       grad_fn=<SelectBackward0>)

In [4]:
np_state_dict = pt_model.state_dict()
np_state_dict = {k: np_state_dict[k].numpy() for k in np_state_dict}

In [28]:
np.array_equal(np_state_dict['patch_embed.proj.bias'] ,tf_model.projection.layers[0].bias.numpy())

True

In [64]:
i = 1
for weight in tf_model.weights:
    i +=1
    if "dense" in weight.name:
        
        isEqual = np.array_equal(weight.numpy(),np_state_dict[tf_pt[weight.name]].transpose() )
    else:
        isEqual = np.array_equal(weight.numpy(),np_state_dict[tf_pt[weight.name]] )

    if not isEqual:
        print(weight.name, weight.numpy().shape , np_state_dict[tf_pt[weight.name]].shape )
i

conv_projection/kernel:0 (2, 4, 4, 3, 96) (96, 3, 2, 4, 4)


184

In [56]:
import json

tf_pt = {}
  
# Opening JSON file
f = open('data.json')
  
# returns JSON object as 
# a dictionary
data = json.load(f)
f.close()
# Iterating through the json
# list
for idx, layer in enumerate(data):
    # print(idx, layer, "------" ,  data[layer])
    tf_pt[data[layer]] = layer
  
# Closing file


# training

In [ ]:
model = tf.keras.Sequential()
model.add(SwinTransformer3D())
model.add(tf.keras.layers.Dense(6))
model.compile(optimizer='sgd', loss='mse')
# This builds the model for the first time:
model.fit(x, y, batch_size=10, epochs=10)

In [ ]:
x = tf.random.normal([100,8, 32,32, 3])
y = tf.random.uniform(shape=[100], minval=0, maxval=5, dtype='int64')
y

In [ ]:
x  = swin(x)
#print(x.shape)

In [ ]:
# swin.summary()

In [ ]:
tf.constant(-100, dtype= 'float64')

# projection

In [5]:
import tensorflow as tf
import torch
from keras.layers import Conv3D , LayerNormalization

In [3]:
embed_dim = 96
patch_size = (2,7,7)

projection = tf.keras.Sequential(
[
    Conv3D(
        embed_dim ,kernel_size = patch_size , strides= patch_size , padding="valid", name= "conv_projection"
    )
],
name = "projection"
)   # data_format= "channels_first"

projection.add(LayerNormalization(epsilon=1e-5))

Metal device set to: Apple M1

systemMemory: 8.00 GB
maxCacheSize: 2.67 GB



2022-08-10 09:18:11.553807: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-08-10 09:18:11.554322: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
x_pt = torch.rand( (1,3, 8,224,224)) * 255
x_np = x_pt.numpy()
x_tf = tf.convert_to_tensor(x_np)
x_tf = tf.transpose(x_tf, perm=(0,2,3,4,1))
x_tf = tf.cast(x_tf, dtype= tf.float32)

x_tf.shape

NameError: name 'torch' is not defined